# Interactive LLM Agent

FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks, including coding and math problem-solving.

In this notebook, we demonstrate how to use `PythonAgent` and `UserProxyAgent` to write code and execute the code. Here `PythonAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for the human user to execute the code written by `PythonAgent`, or automatically execute the code. Depending on the setting of `user_interaction_mode` and `max_consecutive_auto_reply`, the `UserProxyAgent` either solicits feedback from the human user or uses auto-feedback based on the result of code execution. For example, when `user_interaction_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `PythonAgent` without doing any additional steps. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `PythonAgent` directly and return the execution results (success or failure and corresponding outputs) to `PythonAgent`.

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [ ]:
# %pip install flaml[autogen]

## Set your API Endpoint

The [`config_list_gpt4_gpt35`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_gpt4_gpt35) function tries to create a list of gpt-4 and gpt-3.5 configurations using Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure Open API key + base.


In [1]:
from flaml import oai

config_list = oai.config_list_gpt4_gpt35()

## Example Task: Create and execute a python script with agents

In the example below, let's see how to use the agents in FLAML to write a python script and execute the script. This process involves constructing a `PythonAgent` to serve as the assistant, along with a `UserProxyAgent` that acts as a proxy for the human user. In this example demonstrated below, when constructing the `UserProxyAgent`,  we select the `human_input_mode` to "NEVER". This means that the `UserProxyAgent` will not solicit feedback from the human user until the limit defined by `max_consecutive_auto_reply` is reached. For the purpose of this example, we've set this limit to 10.

In [6]:
from flaml.autogen.agent.coding_agent import PythonAgent
from flaml.autogen.agent.user_proxy_agent import UserProxyAgent

# create an assistant which is essentially a PythonAgent instance named "coding_agent"
assistant = PythonAgent("coding_agent", request_timeout=600, seed=42, config_list=config_list)
# create a UserProxyAgent instance named "user"
user = UserProxyAgent(
    "user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.rstrip().endswith("TERMINATE"),
)
# the assistant receives a message from the user, which contains the task description
assistant.receive(
    """Create and execute a script to plot a rocket without using matplotlib""",
    user,
)


**** coding_agent received message from user ****

Create and execute a script to plot a rocket without using matplotlib

**** user received message from coding_agent ****

Creating a rocket involves using ASCII characters to display it visually. Here's a simple script to get you started:

```python
# filename: rocket.py
def plot_rocket():
    rocket = '''
          |
         /_\
        /^|^\ 
        //| \\
       // | \\
    '''
    print(rocket)

if __name__ == "__main__":
    plot_rocket()
```

Save this code in a file named `rocket.py` and execute the script. After that, let me know the results.

 >>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...

**** coding_agent received message from user ****

exitcode: 0 (execution succeeded)
Code output: 
          |
         /_        /^|^\ 
        //| \
       // | \
    


**** user received message from coding_agent ****

It seems that there's an issue with the spacing in the rocket drawing. Let's fix that.

```pytho

Next, let's see how to use the agents to first write the generated script to a file and then execute the script in two sessions of conversation between the `PythonAgent` and the `UserProxyAgent`.

In [7]:
# it is suggested to reset the assistant to clear the state if the new task is not related to the previous one.
assistant.reset()
assistant.receive(
    """Create a temp.py file with the following content:
    ```
    print('Hello world!')
    ```""",
    user,
)


**** coding_agent received message from user ****

Create a temp.py file with the following content:
```
print('Hello world!')
```

**** user received message from coding_agent ****

Here is the code to create the temp.py file with the specified content. Please execute this code:

```python
with open('temp.py', 'w') as file:
    file.write("print('Hello world!')")
```

After executing this code, you should have a file named temp.py with the content:

```
print('Hello world!')
```

 >>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...

**** coding_agent received message from user ****

exitcode: 0 (execution succeeded)
Code output: 

**** user received message from coding_agent ****

Great! The temp.py file has been created successfully. Now, you can run this file to see the output. If you need any further assistance, feel free to ask.

TERMINATE


The example above involves code execution. In FLAML, code execution is triggered automatically by the `UserProxyAgent` when it detects an executable code block in a received message and no human user input is provided. This process occurs in a designated working directory, using a Docker container by default. Unless a specific directory is specified, FLAML defaults to the `flaml/autogen/extensions` directory. Users have the option to specify a different working directory by setting the `work_dir` argument when constructing a new instance of the `UserProxyAgent`.

Upon successful execution of the preceding code block, a file named `temp.py` will be created and saved in the default working directory `flaml/autogen/extensions`. Now, let's prompt the assistant to execute the code contained within this file using the following line of code.

In [8]:
assistant.receive("""Execute temp.py""", user)


**** coding_agent received message from user ****

Execute temp.py

**** user received message from coding_agent ****

To execute temp.py, run the following code:

```python
import os

os.system('python temp.py')
```

This code imports the os module and then runs the temp.py file. After executing this code, you should see the output:

Hello world!

 >>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...

**** coding_agent received message from user ****

exitcode: 0 (execution succeeded)
Code output: Hello world!


**** user received message from coding_agent ****

I'm glad that the code execution was successful and you got the desired output! If you need any further help or assistance with another task, feel free to ask.

TERMINATE
